# Bank Churners
**ML Project 2 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonnell

Descriptions: The first part should be a description which models/classifiers/regressors achieved the best 
    results with your datasets datasets, and what you did to improve those results along the way.

    - Through research we found that a called RandomForestClassifier worked the best after testing a couple different model including
    Niave Bayes and DecisionTreeClassifier. We improved our results by using a feauture engineer and found that selecting all of the features
    proved to provide the best results. This was different from our oroginal assumptions that the type of car would be mostly based upon the price,
    size, and fuel of the car.



### Step 1
Import the necessary packages and frameworks.

In [228]:
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from seaborn import load_dataset

from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce

### Step 2
Read in .csv files and save as DataFrames.

In [229]:
data_train_df = pd.read_csv("cars_class_train.csv")#, nrows=86108) 
data_test_df = pd.read_csv("cars_class_test.csv")#, nrows=86108)
data_train_ft = data_train_df.drop('type', axis=1)
data_train_tgt = data_train_df["type"]

### Step 3
Look at the types of data and get a quick look at what each data type looks like. From there we can select the features we want to try and use. Original assumption
was that the type of car would be mostly based upon the price, size, and fuel of the car.


In [230]:
# To see what kind of data we're working with
print(data_test_df.dtypes)
print(data_train_ft.dtypes)
print(data_train_ft.head(5))

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
Unnamed: 0        int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
pa

### Step 5
Convert data from catgeorical to numerical using a binary encoder. From there we want to get the best features for the data to analyze so we use the SelectKBest and the more features we have, the more accurate
the model seems to be from testing.

In [231]:
features = ['url',
           'region',
           'region_url',
           'price',
           'year',
           'manufacturer',
           'model',
           'condition',
           'cylinders',
           'fuel',
           'odometer',
           'title_status',
           'transmission',
           'VIN',
           'drive',
           'size',
           'paint_color',
           'image_url',
           'description',
           'state',
           'lat',
           'long',
           'posting_date'
          ]


data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'], utc=True)
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'])
data_train_ft['posting_date'] = (pd.DatetimeIndex(data_train_ft['posting_date']).day) + (pd.DatetimeIndex(data_train_ft['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_train_ft['posting_date']).year)*365)
data_train_ft['posting_date'] = data_train_ft['posting_date'].astype(float)

data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'], utc=True)
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'])
data_test_df['posting_date'] = (pd.DatetimeIndex(data_test_df['posting_date']).day) + (pd.DatetimeIndex(data_test_df['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_test_df['posting_date']).year)*365)
data_test_df['posting_date'] = data_test_df['posting_date'].astype(float)
data_train_ft = data_train_ft[features]
data_train_ft = data_train_ft.fillna(0)

#data_test_ft = data_test_df
data_test_df = data_test_df[features]
data_test_df = data_test_df.fillna(0)
'''
encoder = LabelEncoder()
data_train_tgt = encoder.fit_transform(data_train_tgt)
print(data_train_tgt)
encoder = ce.TargetEncoder(return_df=True)
data_train_ft = encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = encoder.fit_transform(data_test_df, data_train_tgt)
# print(y_onehot.head(10))
# data_train_tgt = data_train_tgt.iloc[0:(data_test_df.shape[0])]
r = data_train_tgt.shape[0]
'''
print(f'{r}')
def target_encode_multiclass(train, test, train_tgt): #X,y are pandas df and series
    train_tgt=train_tgt.astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(train_tgt)
    y_onehot=enc.transform(train_tgt)
    class_names=y_onehot.columns  #names of onehot encoded columns
    train_obj=train.select_dtypes('object') #separate categorical columns
    train=train.select_dtypes(exclude='object') 
    test_obj=test.select_dtypes('object') #separate categorical columns
    test=test.select_dtypes(exclude='object') 
    for class_ in class_names:
        enc=ce.TargetEncoder()
        enc.fit(train_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(train_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        train=pd.concat([train,temp],axis=1)    #add to original dataset
        r = test_obj.shape[0]
        c = y_onehot[class_].shape[0]
        print(f'{r}, {c}')
        enc.fit(test_obj,y_onehot[class_].iloc[0:(test_obj.shape[0])]) #convert all categorical 
        temp=enc.transform(test_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        test=pd.concat([test,temp],axis=1)    #add to original dataset
    return train, test, y_onehot
data_train_ft, data_test_df, data_train_tgt = target_encode_multiclass(data_train_ft, data_test_df, data_train_tgt)
r, c = data_train_df.shape
print(f'{r}, {c}')
r = data_train_tgt.shape[0]
print(f'{r}')
r, c = data_test_df.shape
print(f'{r}, {c}')
print('Train ft')
print(data_train_ft.dtypes)
print('\n Test df')
print(data_test_df.dtypes)
print('\n Train tgt')
print(data_train_tgt.dtypes)
print('\n Train Ft')
print(data_train_ft.head(5))
print('\n Test df')
print(data_test_df.head(5))
print('\n Train Tgt')
print(data_train_tgt.head(5))

data_train_ft.head()
data_test_df.head()


86108
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
86108, 200916
200916, 25
200916
86108, 227
Train ft
price                    int64
year                   float64
odometer               float64
lat                    float64
long                   float64
                        ...   
size_type_13           float64
paint_color_type_13    float64
image_url_type_13      float64
description_type_13    float64
state_type_13          float64
Length: 227, dtype: object

 Test df
price                    int64
year                   float64
odometer               float64
lat                    float64
long                   float64
                        ...   
size_type_13           float64
paint_color_type_13    float64
image_url_type_13      float64
description_type_13    float64
state_type_13          float64
Length: 227, dtype: object

 Train tgt
type_1     int64

,price,year,odometer,lat,long,posting_date,url_type_1,region_type_1,region_url_type_1,manufacturer_type_1,...,fuel_type_13,title_status_type_13,transmission_type_13,VIN_type_13,drive_type_13,size_type_13,paint_color_type_13,image_url_type_13,description_type_13,state_type_13
0,3500,"2,013.0000","120,000.0000",29.4589,-98.5699,883.0000,0.0975,0.0996,0.0996,0.0804,...,0.0020,0.0019,0.0016,0.0020,0.0021,0.0026,0.0018,0.0019,0.0019,0.0014
1,8500,"2,004.0000","100,864.0000",42.0990,-75.9180,891.0000,0.0975,0.1019,0.1019,0.1113,...,0.0020,0.0019,0.0019,0.0020,0.0021,0.0016,0.0024,0.0019,0.0019,0.0018
2,14997,"2,011.0000","119,561.0000",38.6050,-121.4019,883.0000,0.0975,0.0958,0.0958,0.1026,...,0.0020,0.0019,0.0019,0.0005,0.0014,0.0018,0.0018,0.0005,0.0019,0.0022
3,5450,"2,014.0000","138,000.0000",42.2421,-84.4282,884.0000,0.0975,0.0714,0.0741,0.0988,...,0.0020,0.0019,0.0019,0.0020,0.0020,0.0018,0.0018,0.0019,0.0019,0.0010
4,4500,"2,010.0000","98,000.0000",27.4547,-82.5359,876.0000,0.0975,0.0913,0.0913,0.1015,...,0.0020,0.0019,0.0019,0.0020,0.0020,0.0018,0.0019,0.0019,0.0019,0.0023


### Step 6
Feauture Engineering

ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, 
                                  k=135)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

print(np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()])
print(data_train_ft.columns.value_counts().count())
new_features = np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()]
data_train_ft = data_train_ft[new_features]
data_test_df = data_test_df[new_features]

### Step 6
Look at data before changing all data types to floats, convert all data type to the same type (float) and look at the updated table.

In [232]:
print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft = data_train_ft.astype(float)
data_test_df = data_test_df.astype(float)

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft:


,price,year,odometer,lat,long,posting_date,url_type_1,region_type_1,region_url_type_1,manufacturer_type_1,...,fuel_type_13,title_status_type_13,transmission_type_13,VIN_type_13,drive_type_13,size_type_13,paint_color_type_13,image_url_type_13,description_type_13,state_type_13
0,8995,"2,011.0000","155,815.0000",41.6002,-93.5701,880.0000,0.0969,0.1826,0.1826,0.3738,...,0.0011,0.0018,0.0019,0.0018,0.0001,0.0019,0.0040,0.0018,0.0000,0.0026
1,4500,"1,954.0000","72,953.0000",40.3074,-79.5424,892.0000,0.0969,0.0797,0.0797,0.1526,...,0.0011,0.0018,0.0019,0.0043,0.0080,0.0030,0.0040,0.0018,0.0018,0.0016
2,16495,"2,013.0000",0.0000,35.2468,-120.6676,876.0000,0.0969,0.1180,0.1180,0.0769,...,0.0011,0.0018,0.0019,0.0043,0.0080,0.0003,0.0034,0.0000,0.0000,0.0009
3,14900,"2,014.0000","64,855.0000",35.7489,-78.6434,887.0000,0.0969,0.0868,0.0868,0.0078,...,0.0011,0.0018,0.0019,0.0018,0.0003,0.0003,0.0002,0.0018,0.0018,0.0011
4,35990,"2,019.0000","10,873.0000",43.0172,-82.4510,884.0000,0.0969,0.0543,0.0543,0.2054,...,0.0011,0.0018,0.0019,0.0000,0.0024,0.0019,0.0002,0.0000,0.0018,0.0018
5,12900,"2,012.0000","97,465.0000",39.8264,-76.9991,872.0000,0.0969,0.0752,0.0752,0.0769,...,0.0011,0.0018,0.0019,0.0018,0.0001,0.0019,0.0002,0.0018,0.0018,0.0016
6,6999,"2,014.0000","90,360.0000",41.3900,-83.6511,889.0000,0.0969,0.0978,0.0978,0.1526,...,0.0011,0.0018,0.0019,0.0005,0.0003,0.0019,0.0003,0.0005,0.0018,0.0016
7,34990,"2,011.0000","84,317.0000",40.4688,-74.2817,873.0000,0.0969,0.1011,0.1011,0.2245,...,0.0123,0.0018,0.0019,0.0000,0.0080,0.0030,0.0396,0.0000,0.0018,0.0016
8,8975,"2,012.0000","36,400.0000",40.5829,-73.9679,875.0000,0.0969,0.0320,0.0320,0.0010,...,0.0011,0.0018,0.0019,0.0018,0.0003,0.0030,0.0040,0.0018,0.0018,0.0010
9,4000,"2,008.0000","117,142.0000",41.7262,-71.3347,883.0000,0.0969,0.0930,0.0930,0.0064,...,0.0011,0.0018,0.0019,0.0043,0.0003,0.0030,0.0005,0.0018,0.0018,0.0023


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Columns: 227 entries, price to state_type_13
dtypes: float64(226), int64(1)
memory usage: 348.0 MB


None

data_train_ft:


,price,year,odometer,lat,long,posting_date,url_type_1,region_type_1,region_url_type_1,manufacturer_type_1,...,fuel_type_13,title_status_type_13,transmission_type_13,VIN_type_13,drive_type_13,size_type_13,paint_color_type_13,image_url_type_13,description_type_13,state_type_13
0,"8,995.0000","2,011.0000","155,815.0000",41.6002,-93.5701,880.0000,0.0969,0.1826,0.1826,0.3738,...,0.0011,0.0018,0.0019,0.0018,0.0001,0.0019,0.0040,0.0018,0.0000,0.0026
1,"4,500.0000","1,954.0000","72,953.0000",40.3074,-79.5424,892.0000,0.0969,0.0797,0.0797,0.1526,...,0.0011,0.0018,0.0019,0.0043,0.0080,0.0030,0.0040,0.0018,0.0018,0.0016
2,"16,495.0000","2,013.0000",0.0000,35.2468,-120.6676,876.0000,0.0969,0.1180,0.1180,0.0769,...,0.0011,0.0018,0.0019,0.0043,0.0080,0.0003,0.0034,0.0000,0.0000,0.0009
3,"14,900.0000","2,014.0000","64,855.0000",35.7489,-78.6434,887.0000,0.0969,0.0868,0.0868,0.0078,...,0.0011,0.0018,0.0019,0.0018,0.0003,0.0003,0.0002,0.0018,0.0018,0.0011
4,"35,990.0000","2,019.0000","10,873.0000",43.0172,-82.4510,884.0000,0.0969,0.0543,0.0543,0.2054,...,0.0011,0.0018,0.0019,0.0000,0.0024,0.0019,0.0002,0.0000,0.0018,0.0018
5,"12,900.0000","2,012.0000","97,465.0000",39.8264,-76.9991,872.0000,0.0969,0.0752,0.0752,0.0769,...,0.0011,0.0018,0.0019,0.0018,0.0001,0.0019,0.0002,0.0018,0.0018,0.0016
6,"6,999.0000","2,014.0000","90,360.0000",41.3900,-83.6511,889.0000,0.0969,0.0978,0.0978,0.1526,...,0.0011,0.0018,0.0019,0.0005,0.0003,0.0019,0.0003,0.0005,0.0018,0.0016
7,"34,990.0000","2,011.0000","84,317.0000",40.4688,-74.2817,873.0000,0.0969,0.1011,0.1011,0.2245,...,0.0123,0.0018,0.0019,0.0000,0.0080,0.0030,0.0396,0.0000,0.0018,0.0016
8,"8,975.0000","2,012.0000","36,400.0000",40.5829,-73.9679,875.0000,0.0969,0.0320,0.0320,0.0010,...,0.0011,0.0018,0.0019,0.0018,0.0003,0.0030,0.0040,0.0018,0.0018,0.0010
9,"4,000.0000","2,008.0000","117,142.0000",41.7262,-71.3347,883.0000,0.0969,0.0930,0.0930,0.0064,...,0.0011,0.0018,0.0019,0.0043,0.0003,0.0030,0.0005,0.0018,0.0018,0.0023


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Columns: 227 entries, price to state_type_13
dtypes: float64(227)
memory usage: 348.0 MB


None

### Step 7
Scale the data to get rid of any outliers that would affect the model.

In [233]:
from sklearn.preprocessing import MinMaxScaler
def scale_datasets(x_train, x_test):

  """
  Standard Scale test and train data
  """
  standard_scaler = MinMaxScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

data_train_ft, data_test_df = scale_datasets(data_train_ft, data_test_df)

### Step 8
Create a library of different models to train and track the accuracy of later. Like the feature selection, having a wide variety of models to test out helps us ensure that our predictions are as accurate as possible.

In [234]:
scaler = skpre.StandardScaler()

pipelines_to_try = {}

# add k-NN models with various values of k to models_to_try
#for k in range(1,3,3):
    #pipelines_to_try[f'{k}-NN'] = neighbors.KNeighborsClassifier(n_neighbors=k)

for k in range(36,40,2):
    pipelines_to_try[f'DTC-{k}'] =  tree.DecisionTreeClassifier(max_depth=k)
for k in range(70,120, 30):
    pipelines_to_try[f'RandomForestClassifier-{k}'] = RandomForestClassifier(n_estimators=k)
baseline = dummy.DummyClassifier(strategy="uniform")
pipelines_to_try1 = {}
# scale the data
for model_name in pipelines_to_try:
    pipelines_to_try1[f'std_{model_name}_pipe'] = pipeline.make_pipeline(scaler, 
                                                      pipelines_to_try[model_name])

### Step 9
Use cross-validation to test the accuracy of the models.

In [235]:
accuracy_scores = {}
loo = skms.LeaveOneOut()
for name, model in pipelines_to_try1.items():
    loo = skms.LeaveOneOut()
    print(f'Currently modeling using : {name}')
    scores = skms.cross_val_score(model,
                                  data_train_ft,
                                  data_train_tgt,
                                  #cv=loo,
                                  #n_jobs=-1,
                                  cv=10,
                                  scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[name] = mean_accuracy
    print(f'{name}: {mean_accuracy:.3f}')
# Took 14m 36.9s on my mac

Currently modeling using : std_DTC-36_pipe


### Step 10
Find the pipeline with the highest accuracy and save it for testing.

In [ ]:
best_pipeline_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest pipeline: {best_pipeline_name} (accuracy = {accuracy_scores[best_pipeline_name]:.3f})')
final_pipeline = pipelines_to_try1[best_pipeline_name]

# Best model seems to be RandomForestClassifier (accuracy = 0.831)


Best pipeline: std_RandomForestClassifier-100_pipe (accuracy = 0.903)


### Step 11
Fit and test the highest-scoring model against the testing data. Save the results in submission.csv for uploading to Kaggle.

In [ ]:
fit = final_pipeline.fit(data_train_ft, data_train_tgt)
predictions = fit.predict(data_test_df)
make_submission_df = pd.read_csv("cars_class_test.csv")
print(predictions)
# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')
# write to csv file
make_submission_df['type'] = predictions
make_submission_df.to_csv('submission.csv',sep=',')
# display message
print("Saved predictions to csv file.")

['sedan' 'pickup' 'wagon' ... 'SUV' 'truck' 'sedan']
Saved predictions to csv file.
